In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.5. Cluster Distribution Analyses/1.5.1. Wilcoxon Rank Sum/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.5. Cluster Distribution Analyses/1.5.1. Wilcoxon Rank Sum/Output')
cur_date = "110821"

library(readxl)
library(data.table)
library(ggplot2)
library(factoextra)
library(janitor)
library(dplyr)
library(tidyverse)
library(gridExtra)
library(cluster)
library(vegan)
library(fpc)
library(ggdendro)

#reading in file
cytokines <- data.frame(read_excel("CytokineData_102920.xlsx", sheet = 2))

#consensus cluster assignments 
consensus_clus <- data.frame(read_excel("110821consensus_clusters.xlsx")) 

Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test



Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.6     ✔ purrr   0.3.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr:

Assigning the same baseline consensus clusters to e-cig and cigarette smokers and running wilcoxon rank sum tests to see if they differ. 

# Eigencytokines

In [39]:
#first try
consensus_clus_dfs = drop_na(consensus_clus) %>%
  group_by(Cluster) %>%
  group_split()

#consensus_clus_dfs

clus_A = consensus_clus_dfs[[1]]
clus_B = consensus_clus_dfs[[2]]
clus_C = consensus_clus_dfs[[3]]

#consensus_clus$Cluster == 'A'
#drop_na(consensus_clus[consensus_clus$Cluster == 'A',])

test = cytokines %>%
  filter(Protein %in% clus_A$Cytokine, Compartment != 'Serum', Group == "NS")

#head(test)

#reshape2::dcast(test, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      #column_to_rownames("SubjectID")

,Original_Identifier,Protein,Conc,Conc_pslog2,Compartment,SubjectNo,Group,SubjectID
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,E_C_F_002,IFNg,17.642316,4.220509,ELF,1,NS,NS_1
2,E_C_F_002,IL10,2.873724,1.953721,ELF,1,NS,NS_1
3,E_C_F_002,IL12p70,1.625272,1.392467,ELF,1,NS,NS_1
4,E_C_F_002,TNFa,13.961637,3.903196,ELF,1,NS,NS_1
5,E_C_F_002,IL17D,52.023660,5.728564,ELF,1,NS,NS_1
6,E_C_F_002,IL4,0.000000,0.000000,ELF,1,NS,NS_1


Aggregation function missing: defaulting to length



,IFNg,IL10,IL12p70,IL17D,IL4,TNFa
,<int>,<int>,<int>,<int>,<int>,<int>
NS_1,3,3,3,3,3,3
NS_10,3,3,3,3,3,3
NS_11,3,3,3,3,3,3
NS_12,3,3,3,3,3,3
NS_13,3,3,3,3,3,3
NS_14,3,3,3,3,3,3
NS_2,3,3,3,3,3,3
NS_3,3,3,3,3,3,3
NS_4,3,3,3,3,3,3


In [6]:
#2nd try
consensus_clus_dfs = drop_na(consensus_clus) %>%
  group_by(Cluster) %>%
  group_split()

#consensus_clus_dfs

clus_A = consensus_clus_dfs[[1]]
clus_B = consensus_clus_dfs[[2]]
clus_C = consensus_clus_dfs[[3]]

#consensus_clus$Cluster == 'A'
#drop_na(consensus_clus[consensus_clus$Cluster == 'A',])

cytokines2 = cytokines %>%
  filter(Protein %in% clus_A$Cytokine, Compartment != 'Serum', Group == "NS") %>%
  group_by(Compartment) %>%
  group_split() #NELF, NLF, sputum

#head(cytokines2)

#NELF data first
NELF_cytokines = reshape2::dcast(cytokines2[[1]], SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
    column_to_rownames("SubjectID")
#NLF
NLF_cytokines = reshape2::dcast(cytokines2[[2]], SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
    column_to_rownames("SubjectID")
#sputum
sputum_cytokines = reshape2::dcast(cytokines2[[3]], SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
    column_to_rownames("SubjectID")

cytokines_clus_A2 = rbind(NELF_cytokines, NLF_cytokines, sputum_cytokines)
cytokines_clus_A2

,IFNg,IL10,IL12p70,IL17D,IL4,TNFa
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS_1,4.22050916,1.95372103,1.392466929,5.72856434,0.000000000,3.9031961
NS_10,1.99001811,1.14516180,2.067844989,2.25084078,0.341378844,2.6275898
NS_11,2.65669076,1.34862315,2.207838025,0.72474191,0.138268201,3.0096455
NS_12,3.43968185,1.80195351,2.139792262,0.00000000,0.131820068,4.3541298
NS_13,2.31834012,1.17649823,0.719413863,3.65801990,0.000000000,2.7163564
NS_14,3.43984997,1.80041224,1.238174645,7.26686648,0.000000000,3.5972186
NS_2,4.39470527,2.65211955,1.894209450,7.97648679,0.045301616,4.9708627
NS_3,0.00000000,0.08945805,0.000000000,0.97686777,0.123621184,0.3201832
NS_4,2.63821301,2.02015536,0.866583643,1.26944758,0.063319744,1.9151961


In [28]:
get_eigencytokines = function(smoking_status){
    #first creating separate consensus cluster dfs
    consensus_clus_dfs = drop_na(consensus_clus) %>%
      group_by(Cluster) %>%
      group_split()

    clus_A = consensus_clus_dfs[[1]]
    clus_B = consensus_clus_dfs[[2]]
    clus_C = consensus_clus_dfs[[3]]
    
    # Separating the cytokine data based on consensus cluster dfs
    clus_A_cytokines = cytokines %>%
      #removing serum data since that wasn't used to derive the consensus clusters
      filter(Group == smoking_status, Protein %in% clus_A$Cytokine, Compartment != 'Serum') %>%
      #splitting by compartment: NELF, NLF, sputum
      group_by(Compartment) %>%
      group_split() 
    clus_B_cytokines = cytokines %>%
      filter(Group == smoking_status, Protein %in% clus_B$Cytokine, Compartment != 'Serum') %>%
      group_by(Compartment) %>%
      group_split() 
    clus_C_cytokines = cytokines %>%
      filter(Group == smoking_status, Protein %in% clus_C$Cytokine, Compartment != 'Serum') %>%
      group_by(Compartment) %>%
      group_split() 
    

    # reshaping data 
    reshaped_clus_A_cytokines = data.frame()
    reshaped_clus_B_cytokines = data.frame()
    reshaped_clus_C_cytokines = data.frame()
    for (i in 1:3){
        #reshaping each compartment (ie. NLF, NELF, sputum) dataframe 
        clus_A_df <- reshape2::dcast(clus_A_cytokines[[i]], SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
          column_to_rownames("SubjectID") 
        clus_B_df <- reshape2::dcast(clus_B_cytokines[[i]], SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
          column_to_rownames("SubjectID") 
        clus_C_df <- reshape2::dcast(clus_C_cytokines[[i]], SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
          column_to_rownames("SubjectID")
        
        #concatenating dfs to create 1 df for each cluster
        reshaped_clus_A_cytokines = rbind(reshaped_clus_A_cytokines, clus_A_df)
        reshaped_clus_B_cytokines = rbind(reshaped_clus_B_cytokines, clus_B_df)
        reshaped_clus_C_cytokines = rbind(reshaped_clus_C_cytokines, clus_C_df)
    }

    # the scale function operates down columns 
    # IS IT OKAY TO SCALE ALL THE DATA TOGETHER??
    clus_A_cytokines_scaled <- reshaped_clus_A_cytokines %>% 
      scale() %>% 
      as.data.frame()
    clus_B_cytokines_scaled <- reshaped_clus_B_cytokines %>% 
      scale() %>% 
      as.data.frame()
    clus_C_cytokines_scaled <- reshaped_clus_C_cytokines %>% 
      scale() %>% 
      as.data.frame()
    
    #transpose cytokine data for each compartment
    reshaped_clus_A_cytokines <- as.data.frame(t(reshaped_clus_A_cytokines))
    reshaped_clus_B_cytokines <- as.data.frame(t(reshaped_clus_B_cytokines))
    reshaped_clus_C_cytokines <- as.data.frame(t(reshaped_clus_C_cytokines))

    #PCA on each cluster, eigenvectors are in rotation -- PROBLEM - for some reason had to convert everything to numeric  
    pca_clus_A <- reshaped_clus_A_cytokines %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_clus_B <- reshaped_clus_B_cytokines %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_clus_C <- reshaped_clus_C_cytokines %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    
    #eigenvector dfs of first principal component 
    eigencytokines_clus_A <- data.frame(pca_clus_A$rotation[,"PC1"])
      colnames(eigencytokines_clus_A)[1] <- "ClusterA"
    eigencytokines_clus_B <- data.frame(pca_clus_B$rotation[,"PC1"])
      colnames(eigencytokines_clus_B)[1] <- "ClusterB"
    eigencytokines_clus_C <- data.frame(pca_clus_C$rotation[,"PC1"])
      colnames(eigencytokines_clus_C)[1] <- "ClusterC"

    
    all_eigencytokines = cbind(eigencytokines_clus_A, eigencytokines_clus_B, eigencytokines_clus_C)
    return(all_eigencytokines)
    
}
eigencytokines_NS = get_eigencytokines("NS")
eigencytokines_NS

,ClusterA,ClusterB,ClusterC
,<dbl>,<dbl>,<dbl>
NS_1,-0.212240236,0.071520522,0.016660673
NS_10,-0.191397817,-0.271033173,-0.194757759
NS_11,-0.117311235,-0.178504213,-0.171575150
NS_12,-0.104092733,-0.154569781,-0.189280594
NS_13,-0.217378588,0.005464235,-0.078764458
NS_14,-0.192870770,-0.023430554,0.167371106
NS_2,-0.205584249,0.001125928,-0.075778723
NS_3,-0.131128180,-0.257831022,-0.211000715
NS_4,-0.140916233,-0.085593112,-0.029676619


In [6]:
get_eigencytokines = function(smoking_status){
    # Separating the cytokine data into consensus cluster dfs
    cytokines <- cytokines %>% 
        filter(Group == smoking_status) %>% #only want non-smokers for baseline analysis
        group_by(Compartment) %>% 
        group_split
    NELF <- cytokines[[1]]
    NLF <- cytokines[[2]]
    Serum <- cytokines[[3]]
    Sputum <- cytokines[[4]]

    # reshaping data 
    NELF <- reshape2::dcast(NELF, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    NLF <- reshape2::dcast(NLF, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    Serum <- reshape2::dcast(Serum, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    Sputum <- reshape2::dcast(Sputum, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 

    #background filter eliminating any cytokines that are not expressed in a compartment  
    NLF$I309 <- NULL
    Sputum$I309 <- NULL 

    # the scale function operates across columns 
    NELF_scaled <- NELF %>% 
      scale() %>% 
      as.data.frame()
    NLF_scaled <- NLF %>% 
      scale() %>% 
      as.data.frame()
    Serum_scaled <- Serum %>% 
      scale() %>% 
      as.data.frame()
    Sputum_scaled <- Sputum %>% 
      scale() %>% 
      as.data.frame()

    #transpose cytokine data for each compartment
    NELF <- as.data.frame(t(NELF))
    NLF <- as.data.frame(t(NLF)) 
    Serum <- as.data.frame(t(Serum))
    Sputum <- as.data.frame(t(Sputum)) 

    #renaming first column, grouping and splitting by "Cluster" column
    NELF_clus <- NELF_clus %>% 
      group_by(Cluster) %>% 
      group_split
    NLF_clus <- NLF_clus %>% 
      group_by(Cluster) %>% 
      group_split
    Serum_clus <- Serum_clus %>% 
      group_by(Cluster) %>% 
      group_split
    Sputum_clus <- Sputum_clus %>% 
      group_by(Cluster) %>% 
      group_split

    #making dfs for each cluster for PCA analysis 
    NELF_1 <- NELF_clus[[1]]
    NELF_2 <- NELF_clus[[2]]
    NELF_3 <- NELF_clus[[3]]

    NLF_1 <- NLF_clus[[1]]
    NLF_2 <- NLF_clus[[2]]
    NLF_3 <- NLF_clus[[3]]

    Serum_1 <- Serum_clus[[1]]
    Serum_2 <- Serum_clus[[2]]
    Serum_3 <- Serum_clus[[3]]

    Sputum_1 <- Sputum_clus[[1]]
    Sputum_2 <- Sputum_clus[[2]]
    Sputum_3 <- Sputum_clus[[3]]

    #making df with subjects' cytokine concentration data for each cluster 
    NELF_1 <- NELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NELF_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NELF_2 <- NELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NELF_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NELF_3 <- NELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NELF_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    NLF_1 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NLF_2 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NLF_3 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    Serum_1 <- Serum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Serum_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Serum_2 <- Serum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Serum_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Serum_3 <- Serum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Serum_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    Sputum_1 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Sputum_2 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Sputum_3 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    #PCA on each cluster, eigenvectors are in rotation -- PROBLEM - for some reason had to convert everything to numeric  
    pca_NELF_1 <- NELF_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NELF_2 <- NELF_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NELF_3 <- NELF_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>%   
      prcomp(center = TRUE, scale = TRUE)

    pca_NLF_1 <- NLF_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NLF_2 <- NLF_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NLF_3 <- NLF_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)

    pca_Serum_1 <- Serum_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Serum_2 <- Serum_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Serum_3 <- Serum_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Sputum_1 <- Sputum_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Sputum_2 <- Sputum_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Sputum_3 <- Sputum_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)

    #eigenvector dfs of first principal component 
    eigencytokines_NELF_1 <- data.frame(pca_NELF_1$rotation[,"PC1"])
      colnames(eigencytokines_NELF_1)[1] <- "ClusterA"
    eigencytokines_NELF_2 <- data.frame(pca_NELF_2$rotation[,"PC1"])
      colnames(eigencytokines_NELF_2)[1] <- "ClusterB"
    eigencytokines_NELF_3 <- data.frame(pca_NELF_3$rotation[,"PC1"])
      colnames(eigencytokines_NELF_3)[1] <- "ClusterC"

    eigencytokines_NLF_1 <- data.frame(pca_NLF_1$rotation[,"PC1"])
      colnames(eigencytokines_NLF_1)[1] <- "ClusterA"
    eigencytokines_NLF_2 <- data.frame(pca_NLF_2$rotation[,"PC1"])
      colnames(eigencytokines_NLF_2)[1] <- "ClusterB"
    eigencytokines_NLF_3 <- data.frame(pca_NLF_3$rotation[,"PC1"])
      colnames(eigencytokines_NLF_3)[1] <- "ClusterC"

    eigencytokines_Serum_1 <- data.frame(pca_Serum_1$rotation[,"PC1"])
      colnames(eigencytokines_Serum_1)[1] <- "ClusterA"
    eigencytokines_Serum_2 <- data.frame(pca_Serum_2$rotation[,"PC1"])
      colnames(eigencytokines_Serum_2)[1] <- "ClusterB"
    eigencytokines_Serum_3 <- data.frame(pca_Serum_3$rotation[,"PC1"])
      colnames(eigencytokines_Serum_3)[1] <- "ClusterC"

    eigencytokines_Sputum_1 <- data.frame(pca_Sputum_1$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_1)[1] <- "ClusterA"
    eigencytokines_Sputum_2 <- data.frame(pca_Sputum_2$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_2)[1] <- "ClusterB"
    eigencytokines_Sputum_3 <- data.frame(pca_Sputum_3$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_3)[1] <- "ClusterC"

    #collapse all eigencytokine dfs
    eigencytokines_NELF <- cbind(eigencytokines_NELF_1, eigencytokines_NELF_2, eigencytokines_NELF_3)
    eigencytokines_NLF <- cbind(eigencytokines_NLF_1, eigencytokines_NLF_2, eigencytokines_NLF_3)
    eigencytokines_Serum <- cbind(eigencytokines_Serum_1, eigencytokines_Serum_2, eigencytokines_Serum_3)
    eigencytokines_Sputum <- cbind(eigencytokines_Sputum_1, eigencytokines_Sputum_2, eigencytokines_Sputum_3)
    
    all_eigencytokines = cbind(eigencytokines_NLF, eigencytokines_NELF, eigencytokines_Sputum, eigencytokines_Serum)
    return(all_eigencytokines)
}

#calling fn
eigencytokines_NLF_NS = get_eigencytokines("NS")[1:3]
eigencytokines_NELF_NS = get_eigencytokines("NS")[4:6]
eigencytokines_Sputum_NS = get_eigencytokines("NS")[7:9]
eigencytokines_Serum_NS = get_eigencytokines("NS")[10:12]
eigencytokines_NLF_Ecig = get_eigencytokines("Ecig")[1:3]
eigencytokines_NELF_Ecig = get_eigencytokines("Ecig")[4:6]
eigencytokines_Sputum_Ecig = get_eigencytokines("Ecig")[7:9]
eigencytokines_Serum_Ecig = get_eigencytokines("Ecig")[10:12]
eigencytokines_NLF_CS = get_eigencytokines("CS")[1:3]
eigencytokines_NELF_CS = get_eigencytokines("CS")[4:6]
eigencytokines_Sputum_CS = get_eigencytokines("CS")[7:9]
eigencytokines_Serum_CS = get_eigencytokines("CS")[10:12]

In [7]:
#exporting eigencytokines these are different than the eigencytokines from the baseline analysis, because these
#now include CS and Ecig users
eigencytokines_NLF = rbind(eigencytokines_NLF_NS, eigencytokines_NLF_CS, eigencytokines_NLF_Ecig)
eigencytokines_NELF = rbind(eigencytokines_NELF_NS, eigencytokines_NELF_CS, eigencytokines_NELF_Ecig)
eigencytokines_Sputum = rbind(eigencytokines_Sputum_NS, eigencytokines_Sputum_CS, eigencytokines_Sputum_Ecig)
eigencytokines_Serum = rbind(eigencytokines_Serum_NS, eigencytokines_Serum_CS, eigencytokines_Serum_Ecig)

# write.csv(eigencytokines_NLF, paste0(Output,"/", cur_date, "_NLF_eigencytokines.csv"), row.names = TRUE)
# write.csv(eigencytokines_NELF, paste0(Output,"/", cur_date, "_NELF_eigencytokines.csv"), row.names = TRUE)
# write.csv(eigencytokines_Sputum, paste0(Output,"/", cur_date, "_Sputum_eigencytokines.csv"), row.names = TRUE)
# write.csv(eigencytokines_Serum, paste0(Output,"/", cur_date, "_Serum_eigencytokines.csv"), row.names = TRUE)

# Wilcoxon Rank Sum tests

Comparing eigencytokines of non-smokers to smokers. 

In [8]:
#converting subject ids to col, melting, and adding compartment
changed_df = function(df, compartment_name){
    df = reshape2::melt(df %>%
        rownames_to_column(var = "SubjectID"), variable = "Cluster",  value.name = 'Conc_pslog2')
    df$Compartment = rep(compartment_name, times = length(df$SubjectID))
    return(df)
}
NS_eigencytokines_NLF = changed_df(eigencytokines_NLF_NS,"NLF")
NS_eigencytokines_NELF = changed_df(eigencytokines_NELF_NS,"NELF")
NS_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_NS,"Sputum")
NS_eigencytokines_Serum = changed_df(eigencytokines_Serum_NS,"Serum")
Ecig_eigencytokines_NLF = changed_df(eigencytokines_NLF_Ecig,"NLF")
Ecig_eigencytokines_NELF = changed_df(eigencytokines_NELF_Ecig,"NELF")
Ecig_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_Ecig,"Sputum")
Ecig_eigencytokines_Serum = changed_df(eigencytokines_Serum_Ecig,"Serum")
CS_eigencytokines_NLF = changed_df(eigencytokines_NLF_CS,"NLF")
CS_eigencytokines_NELF = changed_df(eigencytokines_NELF_CS,"NELF")
CS_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_CS,"Sputum")
CS_eigencytokines_Serum = changed_df(eigencytokines_Serum_CS,"Serum")

#recombining into 3 dataframes
NS_eigencytokines = rbind(NS_eigencytokines_NLF, NS_eigencytokines_NELF, NS_eigencytokines_Sputum, 
                          NS_eigencytokines_Serum)
CS_eigencytokines = rbind(CS_eigencytokines_NLF, CS_eigencytokines_NELF, CS_eigencytokines_Sputum, 
                          CS_eigencytokines_Serum)
Ecig_eigencytokines = rbind(Ecig_eigencytokines_NLF, Ecig_eigencytokines_NELF, Ecig_eigencytokines_Sputum, 
                          Ecig_eigencytokines_Serum)

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables



In [9]:
#creating vectors to loop through
compartment = c('NLF','NELF','Sputum','Serum')
cluster = unique(NS_eigencytokines_NLF$Cluster)

#initializing vectors to store values
CS_df = data.frame()
Ecig_df = data.frame()
wilcoxon_rank_sum_values = function(df1, df2, empty_df){
    #running wilcoxon rank sum and storing the statistic, compartment, cluster, and p value in a vector
    for (i in 1:length(compartment)){
        for (j in 1: length(cluster)){
            variable1_df = df1 %>% # baseline df
                filter(Compartment == compartment[i], Cluster == cluster[j]) %>%
                select(Compartment, Cluster, Conc_pslog2)
            variable2_df = df2 %>% # smoker df
                filter(Compartment == compartment[i], Cluster == cluster[j]) %>%
                select(Compartment, Cluster, Conc_pslog2)


            #running wilcoxon rank sum
            wilcox_test = wilcox.test(variable1_df$Conc_pslog2, variable2_df$Conc_pslog2)
            
            #calculating absolute difference
            AD = (mean(variable2_df$Conc_pslog2) - mean(variable1_df$Conc_pslog2))

            #contains compartment, cluster, u stat, p value
            values_df = cbind(compartment[i], as.character(cluster[j]), AD, wilcox_test$statistic, wilcox_test$p.value)
            empty_df = rbind(empty_df, values_df)

        }
    }
    colnames(empty_df) = c("Compartment",'Cluster', 'AD','Stat', 'P Value')
    return(empty_df)
}

#calling fn
CS_wilcoxon_values = wilcoxon_rank_sum_values(NS_eigencytokines, CS_eigencytokines, CS_df)
Ecig_wilcoxon_values = wilcoxon_rank_sum_values(NS_eigencytokines, Ecig_eigencytokines, Ecig_df)

In [13]:
final_table = function(df){
    #"""
    #Adding a padj col
    
    #:param: vector, demographic variable
    #:output: a 6x12 df containing compartment, cluster, protein, u stat, p value, p adj

    #"""
    
    PAdj = c()
    for (i in 1:length(compartment)){
        single_compartment_df = df %>%
            filter(Compartment == compartment[i])
        padj =  p.adjust(as.numeric(as.character(single_compartment_df$`P Value`)), method = "fdr") 
        PAdj = c(PAdj, padj)
        
    }
    
    df = cbind(df, PAdj)
    return(df)
}

#calling fn
CS_table = final_table(CS_wilcoxon_values)
CS_table
Ecig_table = final_table(Ecig_wilcoxon_values)
Ecig_table

,Compartment,Cluster,AD,Stat,P Value,PAdj
,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
W,NLF,ClusterA,-0.53906101971814,182,9.97093472527582e-08,2.991280e-07
W1,NLF,ClusterB,0.0119333767804518,83,0.720314184153193,7.203142e-01
W2,NLF,ClusterC,-0.123670513346295,119,0.185184786347796,2.777772e-01
W3,NELF,ClusterA,0.00619092076850924,74,0.429599517406759,8.299838e-01
W4,NELF,ClusterB,-0.0136233084574024,96,0.829983797231071,8.299838e-01
W5,NELF,ClusterC,-0.0106803302321949,98,0.756376761739529,8.299838e-01
W6,Sputum,ClusterA,0.0391139960871419,89,0.942952393772154,9.429524e-01
W7,Sputum,ClusterB,0.519577398194044,0,9.97093472527582e-08,2.991280e-07
W8,Sputum,ClusterC,-0.0148431078782217,114,0.279954033990917,4.199311e-01


,Compartment,Cluster,AD,Stat,P Value,PAdj
,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
W,NLF,ClusterA,-0.0226870369614262,191,0.00346913507969652,5.203703e-03
W1,NLF,ClusterB,-0.493596987425873,238,7.54197509809517e-09,2.262593e-08
W2,NLF,ClusterC,-0.122510267739814,133,0.597070760978688,5.970708e-01
W3,NELF,ClusterA,-0.0202826984802277,167,0.0586119692464652,1.758359e-01
W4,NELF,ClusterB,0.0224998516702876,85,0.186378706515446,2.621173e-01
W5,NELF,ClusterC,0.0128519141367842,90,0.262117317119595,2.621173e-01
W6,Sputum,ClusterA,0.0246558747804312,126,0.799302563394779,7.993026e-01
W7,Sputum,ClusterB,0.0257790439966781,87,0.214531813512221,3.217977e-01
W8,Sputum,ClusterC,0.0193029611027928,84,0.173340434102888,3.217977e-01


In [7]:
#exporting tables
write.csv(CS_table, paste0(Output,"/", cur_date, "_CS_Distribution_Analysis.csv"), row.names = FALSE)
write.csv(Ecig_table, paste0(Output,"/", cur_date, "_Ecig_Distribution_Analysis.csv"), row.names = FALSE)